In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../models/ocsvm_df_test_fsborda_pred.csv',sep=';', index_col=0)

In [3]:
df['score_samples'] = df['score_samples'] *-1

A score close to 1 indicates anomalies.  
Score much smaller than 0.5 indicates normal observations.  
If all scores are close to 0.5 then the entire sample does not seem to have clearly distinct anomalies.  

prob_0	= probabilidade de estar satisfeito  
prob_1 = probabilidade de churn

In [4]:
def ks(data=None,target=None, prob=None):
    ## finding at: https://www.listendata.com/2019/07/KS-Statistics-Python.html
    data['target0'] = 1 - data[target]
    data['bucket'] = pd.qcut(data[prob], 5)
    grouped = data.groupby('bucket', as_index = False)
    kstable = pd.DataFrame()
    kstable['min_prob'] = grouped.min()[prob]
    kstable['max_prob'] = grouped.max()[prob]
    kstable['events']   = grouped.sum()[target]
    kstable['nonevents'] = grouped.sum()['target0']
    kstable = kstable.sort_values(by="min_prob", ascending=False).reset_index(drop = True)
    kstable['event_rate'] = (kstable.events / data[target].sum()).apply('{0:.2}'.format).astype(float) * 100
    kstable['nonevent_rate'] = (kstable.nonevents / data['target0'].sum()).apply('{0:.2}'.format).astype(float) * 100
    kstable['cum_eventrate']=(kstable.events / data[target].sum()).cumsum()
    kstable['cum_noneventrate']=(kstable.nonevents / data['target0'].sum()).cumsum()
    kstable['KS'] = np.round(kstable['cum_eventrate']-kstable['cum_noneventrate'], 3) * 100

    #Formating
    kstable['cum_eventrate']= kstable['cum_eventrate'].apply('{0:.2}'.format).astype(float) * 100
    kstable['cum_noneventrate']= kstable['cum_noneventrate'].apply('{0:.2}'.format).astype(float) * 100
    kstable.index = range(1,6)
    kstable.index.rename('Decile', inplace=True)
    pd.set_option('display.max_columns', 9)
    # print(kstable)
    
    #Display KS
    from colorama import Fore
    print(Fore.RED + "KS is " + str(max(kstable['KS']))+"%"+ " at decile " + str((kstable.index[kstable['KS']==max(kstable['KS'])][0])))
    return(kstable)

In [5]:
from scipy.stats import ks_2samp
yhat = df['score_samples']
y = df['TARGET']
def ks_stat(y, yhat):
    return ks_2samp(yhat[y==1], yhat[y!=1]).statistic
ks_stat(y, yhat)

0.3813903445890972

In [6]:
results_quintil = ks(df, target='TARGET', prob='score_samples')
results_quintil

KS is 33.2% at decile 1


,min_prob,max_prob,events,nonevents,event_rate,nonevent_rate,cum_eventrate,cum_noneventrate,KS
Decile,,,,,,,,,
1,1739.063425,182584.589151,315,2726,52.0,19.0,52.0,19.0,33.2
2,69.333678,1738.431432,118,2923,19.0,20.0,71.0,39.0,32.6
3,-1392.738405,67.960450,73,2967,12.0,20.0,83.0,59.0,24.3
4,-2401.917073,-1393.920233,70,2971,12.0,20.0,95.0,79.0,15.5
5,-219872.170103,-2402.377152,31,3010,5.1,21.0,100.0,100.0,0.0


In [7]:
from sklearn.metrics import roc_auc_score

print("Roc Auc: ",roc_auc_score(df['TARGET'], df['score_samples']))
print("Gini: ",2*(roc_auc_score(df['TARGET'], df['score_samples']))-1)

Roc Auc:  0.7144387954510749
Gini:  0.42887759090214983


In [8]:
results_quintil.columns = ['min_prob','max_prob','Qtd de eventos (Target)','Qtd de não-eventos (Target)','% de eventos no Decil','% de não-eventos no Decil','% de evento acumulados','% de não-evento acumulados','KS']
results_quintil.to_csv("results_model10_ocsvm_fs4.csv",sep=';')